Philippe Joly 2025

# Exposition to Symbolic regression
## A ground up implementation of a genetic programming approach to symbolic regression

This serves as a complete implementation and illustration of a simple symbolic regression (SR)algorithm based on the genetic programming (GP) like mutations, crossovers, and natural selection. The notebook relies on the [**PySR**](https://arxiv.org/abs/2305.01582) library by Miles Cranmer and the [symbolic-regression-python](https://github.com/datarobot-community/symbolic-regression-python/tree/master) repository from *datarobot-community*

### Setup

In [43]:
import operator
import numpy as np
import matplotlib.pyplot as plt

### Generate Functions

Generate randomly distributed data from functions. These will be the functions we will want to infer!

In [44]:
# function definitions

## 1 dimensional functions
def f1_1(x, a, b, *args):
    return [a*x +b, f"{a}*x +{b}"]
def f1_2(x, a, b, c, *args):
    return [a*x**2 + b*x + c, f"{a}*x**2 + {b}*x + {c}"]
def f1_3(x, a, b, c, d, *args):    
    return [a*x**3 + b*x**2 + c*x + d, f"{a}*x**3 + {b}*x**2 + {c}*x + {d}"]
def f1_4(x, a, b, c, d, *args):
    return [a*np.sin(b*x+c)+d, f"{a}*sin({b}*x + {c}) + {d}"]
def f1_5(x, a, b, c, *args):
    return [a*np.exp(b*x)+c, f"{a}*exp({b}*x) + {c}"]

## 2 dimensional functions
def f2_1(x, y, a, b, c, *args):
    return [a*x + b*y + c, f"{a}*x + {b}*y + {c}"]
def f2_2(x, y, a, b, c, d, *args):
    return [a*x**2 + b*y**2 + c*x*y + d, f"{a}*x**2 + {b}*y**2 + {c}*x*y + {d}"]
def f2_3(x, y, a, b, c, d, e, *args):
    f = [f1_5(x, a, b, c), f1_5(y, d, e, 0)]
    return [f[0][0] + f[1][0], f[0][1] + "+"+ f[1][1]]
def f2_4(x, y, a, b, c, d, e, *args):
    return [a*np.sin(b*x)*np.exp(c*y)+d*x**2+e*x*y, f"{a}*sin({b}*x)*exp({c}*y) + {d}*x**2 + {e}*x*y"]

## 3 dimensional functions  
def f3_1(x, y, z, a, b, c, d, *args):
    return [a*x + b*y + c*z + d, f"{a}*x + {b}*y + {c}*z + {d}"]
def f3_2(x, y, z, a, b, *args):
    return [a*x**2 + b*y**2 + np.exp(z*y), f"{a}*x**2 + {b}*y**2 + exp(z*y)"]

fxns = [
    [f1_1, f1_2, f1_3, f1_4, f1_5], 
    [f2_1, f2_2, f2_3, f2_4], 
    [f3_1, f3_2]
    ]


In [45]:
def generate_data(fxns, num_pts, num_per_f, sigma):
    x = np.linspace(-10, 10, num_pts[0])

    x2 = np.linspace(-10, 10, np.int16(np.sqrt(num_pts[1])))
    x2, y2 = np.meshgrid(x2,x2)

    x3 = np.linspace(-10, 10, np.int16(np.cbrt(num_pts[2])))
    x3, y3, z3 = np.meshgrid(x3,x3,x3) 

    data = []

    for num_input in range(len(fxns)):
        data.append([])
        for f in fxns[num_input]:
            for i in range(num_per_f):
                # generate random coefficients
                a, b, c, d, e = np.random.randint(-10, 11, 5)

                # generate data from functions
                if num_input == 0:
                    data[-1].append(f(x, a, b, c, d, e))
                elif num_input == 1:
                    data[-1].append(f(x2, y2, a, b, c, d, e))
                elif num_input == 2:
                    data[-1].append(f(x3, y3, z3, a, b, c, d, e))

                # Add gaussion noise
                noise = np.random.normal(loc=0, scale=sigma, size=data[-1][-1][0].shape)
                data[-1][-1][0] += noise

    return ([[x], [x2, y2], [x3, y3, z3]], data)

def print_fxns(data):
    for i, d in enumerate(data):
        print(f"Dimension {i+1}:")
        for j, f in enumerate(d):
            print(f"Function {j+1}: {f[1]}")
        print()

In [46]:
X, data = generate_data(fxns, num_pts=[100, 10000, 1000000], num_per_f=3, sigma=0.1)

print_fxns(data)

Dimension 1:
Function 1: -3*x +-6
Function 2: 6*x +2
Function 3: 1*x +1
Function 4: 5*x**2 + 5*x + -10
Function 5: 9*x**2 + 4*x + -3
Function 6: -2*x**2 + -6*x + 9
Function 7: -9*x**3 + -2*x**2 + -9*x + 8
Function 8: -10*x**3 + -10*x**2 + 5*x + -2
Function 9: -5*x**3 + 2*x**2 + -6*x + 3
Function 10: 9*sin(8*x + -5) + 7
Function 11: 2*sin(-1*x + 0) + 8
Function 12: -7*sin(9*x + -4) + 8
Function 13: 5*exp(-7*x) + 10
Function 14: 7*exp(9*x) + 3
Function 15: 10*exp(2*x) + 7

Dimension 2:
Function 1: -9*x + -9*y + -2
Function 2: 3*x + -7*y + -8
Function 3: -9*x + -3*y + -2
Function 4: 7*x**2 + 1*y**2 + -4*x*y + 3
Function 5: 4*x**2 + -1*y**2 + 7*x*y + 9
Function 6: 6*x**2 + -6*y**2 + 4*x*y + 0
Function 7: 5*exp(9*x) + 3+10*exp(-1*x) + 0
Function 8: -5*exp(3*x) + 6+6*exp(-5*x) + 0
Function 9: -6*exp(3*x) + 5+-6*exp(5*x) + 0
Function 10: -9*sin(9*x)*exp(10*y) + -1*x**2 + -10*x*y
Function 11: 5*sin(-4*x)*exp(7*y) + -6*x**2 + 4*x*y
Function 12: 6*sin(7*x)*exp(-4*y) + 4*x**2 + 6*x*y

Dimension 3